# Testing the lambda handler

In [1]:
from lambda_function import lambda_handler, open_icechunk_repo
import earthaccess
import os
import xarray as xr
import zarr

import warnings
warnings.filterwarnings("ignore") 

In [2]:
# # os.environ['SECRET_ARN'] = 'arn:aws:secretsmanager:us-west-2:444055461661:secret:mursst_lambda_edl_credentials-9dKy1C'
# os.environ['SECRET_ARN'] = 'arn:aws:secretsmanager:us-east-2:444055461661:secret:edl-login-julius-ogbYjL'
# os.environ['DRY_RUN'] = "true"
# TODO: Need to figure out secrets management, for now Ill ingest the env variables manually (see also commented out code in `lambda_function.py`


In [3]:
# #Testing the store bucket with EDL auth (does not work)

# from lambda_function import get_icechunk_creds
# test_creds = get_icechunk_creds()
# import fsspec
# fs = fsspec.filesystem('s3', key=test_creds.access_key_id, secret=test_creds.secret_access_key, token=test_creds.session_token)
# fs.ls('nasa-eodc-public')

In [4]:
#Testing the store bucket with env auth (works)
import fsspec
fs = fsspec.filesystem('s3', )
fs.ls('nasa-eodc-public')

['nasa-eodc-public/ASTER_GDEM',
 'nasa-eodc-public/BlueFlux',
 'nasa-eodc-public/GPM_3IMERGDF.07',
 'nasa-eodc-public/MCD12Q1.061',
 'nasa-eodc-public/MUR-JPL-L4-GLOB-v4.1',
 'nasa-eodc-public/NLDAS',
 'nasa-eodc-public/NLDAS3',
 'nasa-eodc-public/NOAA_FCDR',
 'nasa-eodc-public/cmip6',
 'nasa-eodc-public/icechunk',
 'nasa-eodc-public/oco3',
 'nasa-eodc-public/sample_hdf5_files']

In [5]:
lambda_handler({})

Received event: {}
earthaccess.login()
earthaccess.get_s3_credentials
opening icechunk repo
repo_config {'storage': S3Storage(bucket=nasa-eodc-public, prefix=icechunk/icechunk/MUR-JPL-L4-GLOB-v4.1-virtual-v1-p2, config=S3Options(region=None, endpoint_url=None, anonymous=false, allow_http=false, force_path_style=false)), 'config': RepositoryConfig(inline_chunk_threshold_bytes=None, get_partial_values_concurrency=None, compression=None, caching=None, storage=None, manifest=None), 'virtual_chunk_credentials': {'s3': <builtins.PyCredentials_S3 object at 0x7ff60bf83db0>}}


IcechunkError:   x error getting object from object store service error
  | 
  | context:
  |    0: icechunk::storage::s3::get_ref
  |            with ref_key="branch.main/ref.json"
  |              at icechunk/src/storage/s3.rs:581
  |    1: icechunk::refs::fetch_branch
  |            with name="main"
  |              at icechunk/src/refs.rs:385
  |    2: icechunk::refs::fetch_branch_tip
  |            with name="main"
  |              at icechunk/src/refs.rs:404
  |    3: icechunk::repository::exists
  |              at icechunk/src/repository.rs:316
  |    4: icechunk::repository::open
  |              at icechunk/src/repository.rs:225
  | 
  |-> error getting object from object store service error
  |-> service error
  |-> unhandled error (SignatureDoesNotMatch)
  `-> Error { code: "SignatureDoesNotMatch", message: "The request signature we calculated does not match the signature you provided. Check your key and signing method.", s3_extended_request_id:
      "WxhFQ0mjtcoqG/Dhr9/fA

# Testing the store

In [9]:
earthaccess.login()
ea_creds = earthaccess.get_s3_credentials(daac='PODAAC')
bucket = 'nasa-eodc-public'
store_name = "MUR-JPL-L4-GLOB-v4.1-virtual-v1-p2"
lat_slice = slice(47.6, 47.7)
lon_slice = slice(-122.4, -122.3)

In [11]:
time_range = ["2025-06-30", "2025-07-02"]

### Get a value from the icechunk store

In [13]:
import icechunk as ic


storage = ic.s3_storage(
    bucket=bucket,
    prefix=f"icechunk/{store_name}",
    anonymous=False,
    from_env=True # cannot auth with EDL
)
config = ic.RepositoryConfig.default()
config.set_virtual_chunk_container(ic.VirtualChunkContainer(
# ic.Repository.open(storage=storage)

# config = icechunk.RepositoryConfig.default()
# config.set_virtual_chunk_container(icechunk.VirtualChunkContainer("s3", "s3://", icechunk.s3_store(region="us-west-2")))

# repo_config = dict(
#     storage=storage,
#     config=config,
# )

# if ea_creds:
#     earthdata_credentials = icechunk.containers_credentials(
#         s3=icechunk.s3_refreshable_credentials(get_credentials=get_icechunk_creds),
#     )
#     repo_config['virtual_chunk_credentials'] = earthdata_credentials
# print('repo_config', repo_config)
# return icechunk.Repository.open(**repo_config)

In [8]:
repo = open_icechunk_repo(bucket, store_name, ea_creds)
session = repo.readonly_session(branch="main")
xds = xr.open_zarr(session.store, zarr_version=3, chunks={}, consolidated=False)

repo_config {'storage': S3Storage(bucket=nasa-eodc-public, prefix=icechunk/MUR-JPL-L4-GLOB-v4.1-virtual-v1-p2, config=S3Options(region=None, endpoint_url=None, anonymous=false, allow_http=false, force_path_style=false)), 'config': RepositoryConfig(inline_chunk_threshold_bytes=None, get_partial_values_concurrency=None, compression=None, caching=None, storage=None, manifest=None), 'virtual_chunk_credentials': {'s3': <builtins.PyCredentials_S3 object at 0x7f9bb39e2a30>}}


IcechunkError:   x error getting object from object store service error
  | 
  | context:
  |    0: icechunk::storage::s3::get_ref
  |            with ref_key="branch.main/ref.json"
  |              at icechunk/src/storage/s3.rs:581
  |    1: icechunk::refs::fetch_branch
  |            with name="main"
  |              at icechunk/src/refs.rs:385
  |    2: icechunk::refs::fetch_branch_tip
  |            with name="main"
  |              at icechunk/src/refs.rs:404
  |    3: icechunk::repository::exists
  |              at icechunk/src/repository.rs:316
  |    4: icechunk::repository::open
  |              at icechunk/src/repository.rs:225
  | 
  |-> error getting object from object store service error
  |-> service error
  |-> unhandled error (SignatureDoesNotMatch)
  `-> Error { code: "SignatureDoesNotMatch", message: "The request signature we calculated does not match the signature you provided. Check your key and signing method.", aws_request_id:
      "XS0RHNT4Y0BWAPF4", s3_extende

In [7]:
slice(*time_range)

slice('2025-06-30', '2025-07-02', None)

In [13]:
(time_range[0] + " 09:00:00", time_range[1] + " 09:00:00")

('2025-06-30 09:00:00', '2025-07-02 09:00:00')

In [12]:
xds

<xarray.Dataset> Size: 7TB
Dimensions:           (time: 403, lat: 17999, lon: 36000)
Coordinates:
  * lat               (lat) float32 72kB -89.99 -89.98 -89.97 ... 89.98 89.99
  * lon               (lon) float32 144kB -180.0 -180.0 -180.0 ... 180.0 180.0
  * time              (time) datetime64[ns] 3kB 2024-06-02T09:00:00 ... 2025-...
Data variables:
    analysed_sst      (time, lat, lon) float64 2TB dask.array<chunksize=(1, 1023, 2047), meta=np.ndarray>
    analysis_error    (time, lat, lon) float64 2TB dask.array<chunksize=(1, 1023, 2047), meta=np.ndarray>
    mask              (time, lat, lon) float32 1TB dask.array<chunksize=(1, 1023, 2047), meta=np.ndarray>
    sea_ice_fraction  (time, lat, lon) float64 2TB dask.array<chunksize=(1, 1023, 2047), meta=np.ndarray>
Attributes: (12/47)
    Conventions:                CF-1.7
    title:                      Daily MUR SST, Interim near-real-time (nrt) p...
    summary:                    A merged, multi-sensor L4 Foundation SST anal...
    references:                 http://podaac.jpl.nasa.gov/Multi-scale_Ultra-...
    institution:                Jet Propulsion Laboratory
    history:                    near real time (nrt) version created at nomin...
    ...                         ...
    project:                    NASA Making Earth Science Data Records for Us...
    publisher_name:             GHRSST Project Office
    publisher_url:              http://www.ghrsst.org
    publisher_email:            ghrsst-po@nceo.ac.uk
    processing_level:           L4
    cdm_data_type:              grid

In [8]:
subset = xds.sel(lat=lat_slice, lon=lon_slice, time='2025-07-01')
print(subset.analysed_sst.shape)
subset.analysed_sst.mean().values

(0, 11, 11)


array(nan)

### Get same value from original data

In [9]:
results = earthaccess.search_data(
    short_name='MUR-JPL-L4-GLOB-v4.1',
    temporal=(time_range[0] + " 09:00:00", time_range[1] + " 09:00:00"),
)

direct_access_links = [granule.data_links(access="direct")[0] for granule in results]

fileset = earthaccess.open(direct_access_links, provider='POCLOUD')

og_ds = xr.open_mfdataset(fileset)

QUEUEING TASKS | :   0%|          | 0/3 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/3 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/3 [00:00<?, ?it/s]

In [11]:
og_subset = og_ds.sel(lat=lat_slice, lon=lon_slice, time=slice(*time_range))
print(og_subset.analysed_sst.shape)
og_subset.analysed_sst.mean().values

(3, 11, 11)


array(286.19357576)

# Deleting data

You can delete data directly if necessary or remove previous commits.

### Option 1: first option resizes the arrays.

In [31]:
session = repo.writable_session(branch="main")
store = session.store
variables = ["analysed_sst", "analysis_error", "sea_ice_fraction", "mask", "time"]
resize = 364
for var in variables:
    # Open your array
    group = zarr.open_group(store)
    array = group[var]
    
    # Truncate the array to remove time indices > 364
    # This effectively removes chunks for indices 365, 366, etc.
    new_shape = list(array.shape)
    new_shape[0] = 364  # Assuming time is the first dimension
    array.resize(new_shape)

/opt/conda/lib/python3.11/site-packages/numcodecs/zarr3.py:145: UserWarning: Numcodecs codecs are not in the Zarr version 3 specification and may not be supported by other zarr implementations.
  super().__init__(**codec_config)


In [38]:
# DANGER!
# session.commit("Removed data for time > 2025-05-31")

'44B9XPA2C0QH6FVT5AF0'

### Option 2: Reset to a previous commit.

First list commits:

In [22]:
[(ancestor.message, ancestor.id) for ancestor in repo.ancestry(branch="main")]

[('Committed data for 2025-07-01 09:00:00 21:00:01 to 2025-07-03 21:00:00.',
  'TSWFKSA0JB7WJK52APSG'),
 ('Committed data for 2025-06-30 09:00:00 21:00:01 to 2025-07-02 21:00:00.',
  'W96ECCA8SNV111SB6KG0'),
 ('Committed data for 2025-06-29 09:00:00 21:00:01 to 2025-07-01 21:00:00.',
  '7CPR33AM6TD4B9CNKBEG'),
 ('Committed data for 2025-06-28 09:00:00 21:00:01 to 2025-06-30 21:00:00.',
  '3WQJD66XCMSAKBVZAMF0'),
 ('Committed data for 2025-06-27 09:00:00 21:00:01 to 2025-06-29 21:00:00.',
  '8Y9Z9XA2B9VF3NVNJ1J0'),
 ('Committed data for 2025-05-31 09:00:00 09:00:00 to 2025-06-28 09:00:00.',
  '7PZK6744FDHXHKPYVX8G'),
 ('Removed data for time > 2025-05-31', '44B9XPA2C0QH6FVT5AF0'),
 ('Commit data 2024-10-01 to 2025-06-04', 'GPRDBT2XK9ZERYSQ7EA0'),
 ('Commit data 2024-09-13 to 2024-09-30', 'CP5PHVT9V88VPZTZ0E00'),
 ('Committed data for 2024-09-12 09:00:00 to 2024-09-12 09:00:00 using 20240912090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1',
  'W7Z0Y2FAGZ8WFPMJYZTG'),
 ('Committed data f

In [24]:
# repo.reset_branch("main", "8Y9Z9XA2B9VF3NVNJ1J0")